In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import PIL

import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Input, UpSampling2D
from keras.models import Sequential, Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [1]:
# declare paths to data
TRAIN_PATH = 'data/train'
VALID_PATH = 'data/validation'
TEST_PATH = 'data/test'

# declare image augmentation related hyperparameters
TARGET_SIZE = (256, 256)
RESCALE = 1.0 / 255
COLOR_MODE = 'grayscale'
BATCH_SIZE = 16
ROTATION = 25
BRIGHTNESS = [0.4, 1.0]

# declare flow related hyper parameters 
EPOCHS = 30
CLASSES = ['Damselflies', 'Dragonflies']
CLASS_MODE = 'categorical'
CHECKPOINT = "../checkpoints/weights.hdf5"

## Image Augmentation Pipeline

* #### Training Data

    1. Noramalize pixel values
    2. Horizontal image flip
    3. Vertical image flip
    4. Rotate image
    5. Adjusted image brightness
    6. Grayscale images
    7. Shuffle
    
    
* #### Validation & Testing data

    1. Noramalize pixel values
    2. Grayscale images

In [ ]:
# randomly flip, and rotate images, adjust brightish, and normalize pixel values
trainGenerator = ImageDataGenerator(rescale=RESCALE, 
                                    horizontal_flip=True,  
                                    vertical_flip=True,
                                    rotation_range=ROTATION,
                                    brightness_range=BRIGHTNESS)  

# only scale the pixel values validation images
validatioinGenerator = ImageDataGenerator(rescale=RESCALE)

# only scale the pixel values test images
testGenerator = ImageDataGenerator(rescale=RESCALE)

In [ ]:
# instanciate train flow
trainFlow = trainGenerator.flow_from_directory(
    TRAIN_PATH,
    target_size = TARGET_SIZE,
    batch_size = BATCH_SIZE,
    classes = CLASSES,
    color_mode = COLOR_MODE,
    class_mode = CLASS_MODE,
    shuffle=True
) 

# instanciate validation flow
validationFlow = validatioinGenerator.flow_from_directory(
    VALID_PATH,
    target_size = TARGET_SIZE,
    batch_size = BATCH_SIZE,
    #classes = CLASSES,
    color_mode = COLOR_MODE,
    class_mode= CLASS_MODE,
    shuffle=False
)

## Define Model Architecture


I tried to create a light-weight version of the VGG16.

Instead of two convolution layers before MaxPooling, 
I limited it to one Convolution layer

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(256 , 256, 1)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu' ),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(256, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(.3),
    Dense(64, activation='relu'),
    Dropout(.3),
    Dense(2, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer='Adam', metrics=['accuracy'])

In [ ]:
# create model checkpoint in case of overfitting
checkpoints = ModelCheckpoint(CHECKPOINT, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# train model
history = model.fit(
    trainFlow,
    validation_data=validationFlow, 
    callbacks=[checkpoints],
    epochs=EPOCHS
)